<a href="https://colab.research.google.com/github/Jeorr/ML/blob/main/Lab2Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import zipfile
from tensorflow.keras import layers, models, utils
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Розархівовуємо zip файл, якщо він існує
zip_path = '/content/drive/MyDrive/ML/cats_and_dogs.zip'
extract_to_path = '/content/Lab2'
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"Файли успішно розархівовано в {extract_to_path}")
else:
    print(f"Файл {zip_path} не знайдено.")

# Підготовка шляхів до наборів даних
train_dir = '/content/Lab2/cats_and_dogs/train'
validation_dir = '/content/Lab2/cats_and_dogs/validation'
test_dir = '/content/Lab2/cats_and_dogs/test'

# Генератор для тренувальних даних
train_generator = utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

# Генератор для валідаційних даних
validation_generator = utils.image_dataset_from_directory(
    validation_dir,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format=None,
    verbose=True,
)

# спроба збудувати модель з нуля провалилася, після багатьох спроб вона так і не змогла демонструвати
# хороші результати. Спробуємо попередньо навчену модель як базову для нашої
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True  # Розморозити всі шари

# Заморозити лише верхні шари для збереження попереднього навчання
for layer in base_model.layers[:100]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])

# Компіляція з меншою швидкістю навчання
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Навчання моделі
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
)

# Передбачення
prediction_based_on_validation_data = model.predict(validation_generator)
# Округлення та перетворення на цілі числа (0 або 1)
prediction_based_on_validation_data = np.round(prediction_based_on_validation_data).flatten()

# Справжні мітки
real_data = np.concatenate([y for x, y in validation_generator], axis=0)

# Обчислення MAE та MSE
mae = mean_absolute_error(real_data, prediction_based_on_validation_data)
mse = mean_squared_error(real_data, prediction_based_on_validation_data)

# Виведення результатів
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")

# оскільки тестови картинки в нас не розбиті по категоріям, перевірити відповідність
# тестовим даним нам доведеться робити вручну
# для генератора тестових даних відключаємо мітки, відключаємо перемішування
test_generator = utils.image_dataset_from_directory(
    test_dir,
    labels=None,
    label_mode=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(150, 150),
    shuffle=False,
    verbose=True,
)

# Передбачення на тестових даних
prediction_based_on_test_data = model.predict(test_generator)

# Інтерпретація результатів
predicted_classes = np.round(prediction_based_on_test_data).flatten()

# Отримання назв файлів
file_names = test_generator.file_paths

# Створюємо DataFrame для результатів
results = pd.DataFrame({
    "FileName": file_names,
    "PredictedClass": ["Dog" if cls == 1 else "Cat" for cls in predicted_classes.flatten()]
})

# Виводимо результати в консоль
print(results)

Файли успішно розархівовано в /content/Lab2
Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.8474 - loss: 0.3120 - val_accuracy: 0.9750 - val_loss: 0.1026
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.9731 - loss: 0.0722 - val_accuracy: 0.9680 - val_loss: 0.1067
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 221s 3s/step - accuracy: 0.9744 - loss: 0.0652 - val_accuracy: 0.9700 - val_loss: 0.1186
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.9884 - loss: 0.0306 - val_accuracy: 0.9700 - val_loss: 0.1248
Epoch 5/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - accuracy: 0.9948 - loss: 0.0210 - val_accuracy: 0.9700 - val_loss: 0.1212
Epoch 6/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 215s 3s/step - accuracy: 0.9912 - loss: 0.0330 - val_accuracy: 0.8880 - val_loss: 0.6020
Epoch 7/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - accuracy